In [34]:
import pyvisa
import os
import numpy as np
import time
from qcodes import VisaInstrument
import array
import struct
import smtplib, ssl
import matplotlib.pyplot as plt

In [60]:
class FridgeHandler:
    def __init__(self):
        self.inst = pyvisa.ResourceManager().open_resource('ASRL1::INSTR')
        print('Fridgeboy object created correctly!\n')
  
    def execute(self, cmd):
        self.inst.write('$'+ str(cmd))

    def read(self, cmd):                #It may happen that the read command returns strange things with ?s and Es. In that case you can't trust the result
        out = '?'
        while ('?' in out[0]) or ('E' in out[0]) or ('A' in out[0]):
            out = self.inst.query_ascii_values(str(cmd), converter='s')
            #print(out)       
        out = str.rstrip(out[0])
        return out

    def get_sensor(self, cmd = 3):
        '''Measure temperature or pressure of a sensor of the system. Default: MC temperature.'''                           
        k = self.read('R' + str(cmd))
        k = k.replace("R", "")
        k = k.replace("+", "")
        k = float(k)
        return k       

    def scan_T(self, cmd, interval, tottime):      # cmd -> command that specifies which temperature,
                                                # interval -> time step to perform control
                                                # time -> total time of the scansion
        '''Temperature scansion that prints the values every "interval" seconds for a "time" time'''
        N = int(tottime/interval)
        Temps = np.zeros(N)
        for i in range(N):
            out = self.inst.query_ascii_values(cmd, converter='s')
            out = str.rstrip(out[0])
            time.sleep(interval)
            out = (out.split('+'))[1]           # split the string where '+' is and gives back only the second part (1)
            Temps[i] = float(out)
            print('Temperature at step ' + str(i) + ' is ' + str(out))
        return Temps

    def state(self):
        out = self.inst.query_ascii_values('X', converter='s')
        out = str.rstrip(out[0])
        print(out)
    
    def set_T(self, T):
        '''Set temperature of the mixing chamber to arbitrary value in 0.1 mK. 
        Be careful! The value of temp has to be specified with 5 figures!
        Range is the command name for the power range (E1, E2 ...)'''
        
        cmd = 'E'
        if T <= 50:
            cmd += '1'
        elif T <= 90:
            cmd += '2'
        elif T <= 140:
            cmd += '3'
        elif T <= 400:
            cmd += '4'
        else:
            cmd += '5'

        self.execute(cmd)
        self.execute('A2')
        self.execute('T' + str(10*int(T)))

    def send_email_yahoo(self, fromMy , to, subj, message_text ):         #Be careful, the 'to' variable has to be an array. Put brackets even if it's a single address.
        date = 1/1/2000
        msg = "From: %s\nTo: %s\nSubject: %s\nDate: %s\n\n%s" % ( fromMy, to, subj, date, message_text )
 
        server = smtplib.SMTP("smtp.mail.yahoo.com",587)
        server.starttls()
        password = str('########')  
        server.login(fromMy,password)
        for account in to:
            server.sendmail(fromMy, account, msg)
        server.quit()    
        print ('ok the emails have been sent ')

    def check_p(self):
        out = self.get_sensor(14) < 100000 and self.get_sensor(15) < 1000000  #!!!!!!!!!!! check the pressure values
        if (not out):
            print("High pressure! O_O' ")
            #self.send_email_yahoo("leomaria2906@yahoo.com",["l.mariani48@campus.unimib.it","a.angeloni3@campus.unimib.it",
            #"m.faggionato1@campus.unimib.it","marco.faverzani@unimib.it"],"High pressure!","la pressione è alta")
        return out  

#Possiamo provare ad implementare un tempo dopo il quale, se la temperatura non è stabile, usciamo dal ciclo?    
    def check_stability(self, T, error, sleeptime = 5, pause = 10):     # T --> desired temperature
                                                                                        # error --> uncertainty allowed on the temperature
                                                                                        # interval --> minimum time of stability required. Defaults to 1 minute and half                                                             # sleeptime --> time interval between each check. Defaults to 5 seconds
        counter = 0
        countermax = 10
        out = False
        while (counter < countermax): 
            '''
            if self.get_sensor(21) < 5:
                self.send_email_yahoo("leomaria2906@yahoo.com",["l.mariani48@campus.unimib.it","a.angeloni3@campus.unimib.it",
                "m.faggionato1@campus.unimib.it","marco.faverzani@unimib.it"],"Low pressure!","la pressione P2 è scesa sotto a 5 mbar") 
                counter = countermax + 1                                                             
            if self.get_sensor(2) > 22000:
                self.send_email_yahoo("leomaria2906@yahoo.com",["l.mariani48@campus.unimib.it","a.angeloni3@campus.unimib.it",
                "m.faggionato1@campus.unimib.it","marco.faverzani@unimib.it"],"High temperature!","la temperatura della 1K Pot ha superato i 2.2 K")
                counter = countermax + 1
                '''
            if (T-error < self.get_sensor() and self.get_sensor() < T + error): # check if values are ok. change get_sensor parameters (actually remove them -> they'll default to mixing chamber) !!!!!!!!
                counter = 0
                print('I found a temperature value out of range. I am going to sleep for ' + str(pause) + ' seconds')
                time.sleep(pause) #sleeps for 10 seconds default minutes if T not stable         
            else:
                counter += 1
                time.sleep(sleeptime) #5 sec of sleep between each T check
        if counter == countermax:
            print("Temperature is stable and fridgeboy is ready!")
            out = True
        return out

In [61]:
frigo = FridgeHandler()

Fridgeboy object created correctly!



In [62]:
frigo.set_T(150)

In [64]:
frigo.scan_T(cmd='R3', interval=4, tottime=300)

KeyboardInterrupt: 